In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_df.drop(["id","target","f_27"], axis=1), train_df.target)

In [ ]:
model = Sequential([
    Dense(512, input_dim=x_train.shape[1], activation="swish"),
    Dense(256, activation="swish"),
    Dense(128, activation="swish"),
    Dense(64, activation="swish"),
    Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss=BinaryCrossentropy(), metrics=[AUC(name="auc")])
lr = ReduceLROnPlateau(
    monitor='val_auc',
    factor=0.5,
    patience=10,
    verbose=1)
es = EarlyStopping(
    monitor='val_auc',
    patience=10,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=4096, callbacks=[lr, es])

In [ ]:
prediction = model.predict(test_df.drop(["id","f_27"],axis=1))

In [ ]:
prediction_df = test_df[["id"]]
prediction_df["target"] = prediction

In [ ]:
prediction_df.to_csv("sub_nn.csv", index=False)

In [ ]:
prediction_df.head()